In [ ]:
import create_dict
import torch
import os
import json

# Clean Data

In [ ]:
folder_name = '/data2'
#create_dict.clean_obj(folder_name)

# Generate Dataset

In [ ]:
input_folder = '/data'
suffix = '_test' #suffix will append data_trainxxxx.json
max_elements_per_class = 5
create_dict.create_dictionary(input_folder, max_elements_per_class, suffix)

# Initialize Network

In [ ]:
import SiameseNet

device = torch.device("cuda:0" if torch.cuda.torch.cuda.is_available() else "cpu")

batch_size = 2
net = SiameseNet.SiameseNet(batch_size)
suffix = '_test' # comment in if not coming from generating the dataset
path_to_params = "models/test.pt" # if file does not exist or is empty it starts from untrained and later saves to the file

# shift to GPU if available

net.to(device)
working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

In [ ]:
device

# Train Network

In [ ]:
#training parameters

writer_suffix = 'testing'
margin = 0.5
num_epochs = 1
print_batch = 1
lr = 0.0001



In [ ]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir)

# Validation

In [ ]:
writer_suffix = 'Val'
margin = 0.5

y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir)

In [ ]:
from PIL import Image
import numpy as np
from matplotlib.pyplot import imshow


with open(data_dir_val, 'r') as fp:
    data_val = json.load(fp)
with open(class_dir, 'r') as fp:
    data_class = json.load(fp)
num_KNN = 2
max_show = 3
for i, key in enumerate(ids):
    if (i>=max_show):
        break
    print("Sample: ID:", key, " Description:", data_val[key][1])
    for j in range(num_KNN):
        ID = ids[y_pred[i][j]]
        class_name = data_class[ID]
        name = str('images/' + class_name + '/' + ID + '/models/model_normalized.png')
        img = Image.open(name)
        imshow(np.asarray(img))